SchedulerAI

In [ ]:
%pip install langchain

In [ ]:
%pip install replicate

In [ ]:
%pip install gcsa

Get Replicate API key and save to env variables

In [5]:
from getpass import getpass

REPLICATE_API_TOKEN = getpass()

In [6]:
import os

open = os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

Import modules

In [7]:
from langchain_community.llms import Replicate
from datetime import datetime

Define LLM

In [10]:
llm = Replicate(
    model="meta/meta-llama-3-70b-instruct",
)

-----

This function takes a text input, prompts for a date and two appointment times, and returns them in a specific format: 'date old_appointment_time new_appointment_time', using today's date as default.

In [13]:
def get_date_time(text):
    dt=datetime.date(datetime.now())
    print(dt)
    prompt = "Get the appointment date in format 'YYYY-MM-DD, get the new appointment time in format 'Hours:Mins' and 24 hour format, get the old appointment time in format 'Hours:Mins' and 24 hour format. If there is no old appointment time then old appointment time is new appointment time. Return only the answer in the format 'date old_appointment_time new_appointment_time'.Do not add any conversation to your answer. For reference, a sample answer is '2024-04-27 13:30 15:00'. The text is"+text+".The date right now is"+str(dt)+". Default date is today"
    response = llm(prompt)
    print("date_time:",response)
    response = response.strip().split(" ")
    return response[0], response[1], response[2]

------

This function prompts for an event type (add, delete, shift, or check appointment) based on the input text and returns the event type.

In [5]:
def get_event(text,llm):
    prompt = f"Get the event type in all lowercase. Events can be 'add appointment', 'delete appointment', 'shift appointment' or 'check appointment'. Return only the answer with no conversation. For reference, a sample answer is 'add'. The text is: {text}"
    response = llm(prompt)
    print("event:",response)
    return response

----

This function prompts for the name of the person who wants an appointment based on the input text and returns the name.

In [6]:
def get_name(text,llm):
    prompt = f"Respond with the name of the person who wants and appointment according to the text: {text}. Answer without any conversation."
    response = llm(prompt)
    print("name:",response)
    return response

---

This code imports necessary modules from the gcsa library for Google Calendar integration, imports the `datetime` module, sets the path to Google API credentials, and initializes a GoogleCalendar object with specific calendar ID and credentials path, enabling access to a Google Calendar.

In [7]:
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar
from beautiful_date import hours
from datetime import datetime

credential_path=".credentials/credentials.json"
calendar = GoogleCalendar("1df9a64aaf07fec4a4769ffe32aa5bf54a8543b3eb2ab82fd683f3942282a5e2@group.calendar.google.com",credentials_path=credential_path, save_token=True)

---

This function attempts to check for an event at a specified date and start time within a Google Calendar. If successful, it returns the event; otherwise, it returns False. If there's an error, it prints "ERROR: Wrong info for check".

In [15]:
def check_event(datei, start_time):
    try:
        datei=datei.strip().split("-")
        year, month, day = int(datei[0]), int(datei[1]), int(datei[2])

        start_time=start_time.strip().split(":")
        hr, minute = int(start_time[0]), int(start_time[1])

        start = datetime(year=year, month=month, day=day, hour=hr, minute=minute)
        end = start + 1 * hours

        events = list(calendar.get_events(time_min=start, time_max=end))

        if len(events)>0:
            print("Event present")
            return events[0]
        print("Event absent")
        return False
    
    except:
        print("ERROR: Wrong info for check")

---

This function attempts to add an event to the Google Calendar for a specified date, start time, and name. It first checks if there's any conflicting event using the `check_event` function. If there's no conflict, it creates a new event object and adds it to the calendar; otherwise, it prints "Time slot already taken". If there's an error, it prints "ERROR: Wrong info for adding".

In [16]:
def add_event_cal(datei,start_time, Name):
    try:
        event=check_event(datei,start_time)
        if event==False:
            datei=datei.strip().split("-")
            year, month, day = int(datei[0]), int(datei[1]), int(datei[2])
            start_time=start_time.strip().split(":")
            hr, minute = int(start_time[0]), int(start_time[1])
            start = datetime(year=year, month=month, day=day, hour=hr, minute=minute)
            end = start + 1 * hours
            event = Event(Name,
                        start=start,
                        end=end)
            calendar.add_event(event)
            print("Event added")
        else:
            print("Time slot already taken")
    except:
        print("ERROR: Wrong info for adding")
        return

---

This function attempts to delete an event from the Google Calendar for a specified date and start time. It first checks if there's an event at the given date and time using the `check_event` function. If there's no event, it prints "No Event to Delete". Otherwise, it deletes the event from the calendar. If there's an error, it prints "ERROR: Wrong info for delete".

In [17]:
def delete_event_cal(datei,start_time):
    try:
        event = check_event(datei,start_time)
        if event==False:
            print("No Event to Delete")
            return
        else:
            calendar.delete_event(event)
            print("Event deleted")
    except:
        print("ERROR: Wrong info for delete")

---

This function attempts to shift the timing of an event in the Google Calendar from an old time to a new time on a specified date. It first checks if there's an event at the old time using the `check_event` function. If there is, it updates the event's start and end times with the new time and updates the event in the calendar using `calendar.update_event(event)`. If there's no event at the old time, it prints "No event to Reschedule". If there's an error, it prints "ERROR: Wrong info for shifting".

In [18]:
def shift_event_cal(datei, old_time, new_time):
    try:
        event = check_event(datei, old_time)
        print(event)
        if event!=False:
            datei=datei.strip().split("-")
            year, month, day = int(datei[0]), int(datei[1]), int(datei[2])

            new_time=new_time.strip().split(":")
            hr, minute = int(new_time[0]), int(new_time[1])
            start = datetime(year=year, month=month, day=day, hour=hr, minute=minute)
            end = start + 1 * hours

            event.start=start
            event.end=end

            calendar.update_event(event)
            print("Event Updated")
        else:
            print("No event to Reschedule")
        
    except:
        print("ERROR: Wrong info for shifting")

---

This function attempts to modify events based on the provided text input. It extracts the date, old time, and new time using the `get_date_time` function, determines the event type using the `get_event` function, and processes the modification accordingly. If the event type is 'add', it prompts for the name and adds the event; if 'delete', it deletes the event; if 'shift', it shifts the event timing; if 'check', it checks for an event. If there's an error during the process, it prints "Error".

In [12]:
def modify(text):
    try:
        date, old_time, new_time = get_date_time(text,llm)
        event = get_event(text,llm)

        if event=='add':
            name = get_name(text,llm)
            if new_time=='-1':
                new_time==old_time
            add_event_cal(date ,new_time, name)
        elif event=='delete':
            if new_time=='-1':
                new_time==old_time
            delete_event_cal(date, new_time)
        elif event=='shift':
            shift_event_cal(date, old_time, new_time)
        elif event=='check':
            if new_time=='-1':
                new_time==old_time
            check_event(date, new_time)
        else:
            print("Error: Event not found")
            return
    except:
        print("Error")

Trial Run Example of the code:

This function aims to delete a 3 PM appointment. It does so by extracting the date, old time, and new time from the input text using the `get_date_time` function. Then, it determines the event type, which is 'delete' in this case, using the `get_event` function. Finally, it attempts to delete the event with the specified time. If there's any error during this process, it prints "Error".

In [13]:
modify("delete 3pm appointment")

C:\Users\shrey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


date_time: 2024-04-28 15:00 15:00
event: delete
No Event to Delete


---

TRY IT YOURSELF!

In [ ]:
text = input()
modify(text)